# DataProcessing

DataMiner module provided us relatively clear datasets. Altough these datasets are from multiple websites and there needs to be done some additional clearing.

This module generates 4 new files (clear_timeseries.json, clear_financials.json, clear_overview.json, clear_time_series_monthly_adjusted.json) which are sufficient for data interpretation.

In [1]:
import json
import copy

from datetime import datetime

First we get historical constituents of indexes (SP500, NASDAQ, DJI).

In [42]:
with open('constituents.json', 'r') as f, \
    open('hist_constituents.json', 'r') as f_hist:
        constituents = json.load(f)
        hist_changes = json.load(f_hist)

def remove_blacklist_symbols(timeseries, blacklist):
    for year, symbols in timeseries.items():
        for symbol in symbols:
            if symbol in blacklist:
                timeseries[year].remove(symbol)
    return timeseries
        
def get_historical_constituents(index, constituents, hist_changes):
    for i in constituents:
        if i['symbol'] == index:
            constit = i['constituents']
            
    for i in hist_changes:
        if i['symbol'] == index:
            changes = i['historicalConstituents']
    
    timeseries = {}
    # Maps symbols which for some reason are or are not in index even though they should be
    blacklist = []
    
    timeseries[datetime.strptime(changes[0]['date'], '%Y-%m-%d').year] = constit
    
    for change in changes:
        year = datetime.strptime(change['date'], '%Y-%m-%d').year
        if year < 2000:
            break
            
        temp_year = year    
        default_val = None
        while default_val == None:
            default_val = copy.deepcopy(timeseries.get(temp_year))
            temp_year += 1
            
        timeseries[year] = timeseries.get(year, default_val)
        symbol_change = change['symbol']
        if change['action'] == 'add':
            if symbol_change not in timeseries[year]:
                blacklist.append(symbol_change)
            else:
                timeseries[year].remove(symbol_change)
        else:
            if symbol_change in timeseries[year]:
                blacklist.append(symbol_change)
            else:
                timeseries[year].append(symbol_change)
                
    # Fills timeseries[no_change_year] where there were no changes in year
    timeseries[2000] = timeseries.get(min(timeseries))
    for year in range(2001, 2021):
        timeseries[year] = timeseries.get(year, timeseries[year-1])
    remove_blacklist_symbols(timeseries, blacklist)
    return timeseries
    
timeseries_all_indexes = {"^GSPC": get_historical_constituents("^GSPC", constituents, hist_changes),
              "^NDX": get_historical_constituents("^NDX", constituents, hist_changes),
              "^DJI": get_historical_constituents("^DJI", constituents, hist_changes)}
print(timeseries_all_indexes["^GSPC"])

{2021: ['COG', 'MPC', 'INCY', 'GPS', 'ROP', 'CPB', 'MSI', 'ORCL', 'BLL', 'EXPD', 'BA', 'LEG', 'ALLE', 'CBRE', 'XOM', 'MDT', 'CAH', 'MHK', 'COP', 'ADBE', 'MO', 'SPGI', 'MSFT', 'ROST', 'TDG', 'MKTX', 'CNP', 'BEN', 'MXIM', 'DLR', 'ROL', 'FITB', 'OMC', 'KEYS', 'KSU', 'BXP', 'PRGO', 'MGM', 'ABMD', 'ANTM', 'O', 'ANET', 'CVX', 'MU', 'BWA', 'KHC', 'PXD', 'DISCA', 'HLT', 'IFF', 'AES', 'LHX', 'CVS', 'MCO', 'AVB', 'SWK', 'NVR', 'SHW', 'DVN', 'NRG', 'FB', 'ACN', 'MTD', 'CAT', 'TTWO', 'CDW', 'GE', 'KMB', 'GRMN', 'MRK', 'WHR', 'IPGP', 'CHRW', 'WDC', 'XEL', 'XRX', 'FLS', 'LB', 'EA', 'ALB', 'ATO', 'EMN', 'TFX', 'UPS', 'APH', 'AAPL', 'LMT', 'GPC', 'HII', 'TGT', 'VZ', 'KMI', 'PHM', 'MLM', 'SBAC', 'INTC', 'DIS', 'CNC', 'UA', 'FTNT', 'PAYC', 'CHTR', 'MAA', 'F', 'UNH', 'AMCR', 'K', 'TPR', 'VRTX', 'ESS', 'AAP', 'CLX', 'BIIB', 'GILD', 'GD', 'HCA', 'AKAM', 'APTV', 'EQIX', 'FBHS', 'WAB', 'NKE', 'XYL', 'AMD', 'DXC', 'IP', 'NUE', 'CME', 'LYB', 'VNT', 'AON', 'PFG', 'FAST', 'GOOG', 'MOS', 'WFC', 'NDAQ', 'LW', 'SNA

Now we will clear our data from symbols that could not be obtained via API.

In [3]:
with open('fail_financials.json', 'r') as f_fin, \
    open('fail_overview.json', 'r') as f_overview, \
    open('fail_time_series_monthly_adjusted.json', 'r') as f_monthly:
    blacklist = []
    blacklist.extend(json.load(f_fin))
    blacklist.extend(json.load(f_overview))
    blacklist.extend(json.load(f_monthly))
    
for timeseries in timeseries_all_indexes.values():
    remove_blacklist_symbols(timeseries, blacklist)
    
with open('clear_timeseries.json', 'w') as f:
    json.dump(timeseries_all_indexes, f)

After this we have clear historical timeseries of indexes constituents saved in clear_timeseries.json

Now we will modify financials.json, overview.json, time_series_monthly_adjusted.json so that company data are accessed by its ticker and financials are accessed by its ticker and report period. Also in time_series_monthly_adjusted.json file we will only save adjusted close price and date

In [19]:
with open('financials.json', 'r') as f:
    financials = json.load(f)

clear_financials = {}
for company in financials:
    try:
        ticker = company['results'][0]['ticker']
    except IndexError:
        continue
    temp_dict = {}
    try:
        for financials_from_year in company['results']:
            year = datetime.strptime(financials_from_year['calendarDate'], '%Y-%m-%d').year
            temp_dict[year] = financials_from_year
    except IndexError:
        print(company)
        continue
    clear_financials[ticker] = temp_dict
        
with open('clear_financials.json', 'w') as f:
    json.dump(clear_financials, f)

In [55]:
clear_financials['IBM'][2019]

{'ticker': 'IBM',
 'period': 'Y',
 'calendarDate': '2019-12-31',
 'reportPeriod': '2019-12-31',
 'updated': '2020-04-28',
 'dateKey': '2020-02-25',
 'accumulatedOtherComprehensiveIncome': -28597000000,
 'assets': 152186000000,
 'assetsAverage': 146846000000,
 'assetsCurrent': 38420000000,
 'assetsNonCurrent': 113766000000,
 'assetTurnover': 0.525,
 'bookValuePerShare': 23.49,
 'capitalExpenditure': -2370000000,
 'cashAndEquivalents': 8313000000,
 'cashAndEquivalentsUSD': 8313000000,
 'costOfRevenue': 40659000000,
 'consolidatedIncome': 9431000000,
 'currentRatio': 1.019,
 'debtToEquityRatio': 6.295,
 'debt': 68158000000,
 'debtCurrent': 10177000000,
 'debtNonCurrent': 57981000000,
 'debtUSD': 68158000000,
 'deferredRevenue': 15877000000,
 'depreciationAmortizationAndAccretion': 6059000000,
 'deposits': 0,
 'dividendYield': 0.046,
 'dividendsPerBasicCommonShare': 6.48,
 'earningBeforeInterestTaxes': 11506000000,
 'earningsBeforeInterestTaxesDepreciationAmortization': 17565000000,
 'EBIT

In [6]:
with open('overview.json', 'r') as f:
    overview = json.load(f)

clear_overview = {}
counter = 0
for company in overview:
    try:
        ticker = company['Symbol']
        clear_overview[ticker] = company
        counter += 1
    except KeyError:
        pass 

with open('clear_overview.json', 'w') as f:
    json.dump(clear_overview, f)

In [7]:
clear_overview['IBM']

{'Symbol': 'IBM',
 'AssetType': 'Common Stock',
 'Name': 'International Business Machines Corporation',
 'Description': "International Business Machines Corporation provides integrated solutions and services worldwide. Its Cloud & Cognitive Software segment offers software for vertical and domain-specific solutions in health, financial services, and Internet of Things (IoT), weather, and security software and services application areas; and customer information control system and storage, and analytics and integration software solutions to support client mission critical on-premise workloads in banking, airline, and retail industries. It also offers middleware and data platform software, including Red Hat that enables the operation of clients' hybrid multi-cloud environments; and Cloud Paks, WebSphere distributed, and analytics platform software, such as DB2 distributed, information integration, and enterprise content management, as well as IoT, Blockchain and AI/Watson platforms. The 

In [47]:
with open('time_series_monthly_adjusted.json', 'r') as f:
    time_series_monthly_adjusted = json.load(f)

clear_time_series_monthly_adjusted = {}
for company in time_series_monthly_adjusted:
    try:
        ticker = company['Meta Data']['2. Symbol']
        temp_dict = {}
        for date, price in company['Monthly Adjusted Time Series'].items():
            temp_dict[date] = price['5. adjusted close']
        clear_time_series_monthly_adjusted[ticker] = temp_dict
    except KeyError:
        pass 
    
with open('clear_time_series_monthly_adjusted.json', 'w') as f:
    json.dump(clear_time_series_monthly_adjusted, f)

In [56]:
clear_time_series_monthly_adjusted['IBM']

{'2021-02-08': '123.6100',
 '2021-01-29': '119.1100',
 '2020-12-31': '125.8800',
 '2020-11-30': '123.5200',
 '2020-10-30': '110.1065',
 '2020-09-30': '119.9773',
 '2020-08-31': '121.5944',
 '2020-07-31': '119.6686',
 '2020-06-30': '117.5564',
 '2020-05-29': '121.5765',
 '2020-04-30': '120.5974',
 '2020-03-31': '106.5456',
 '2020-02-28': '125.0060',
 '2020-01-31': '136.6067',
 '2019-12-31': '127.3969',
 '2019-11-29': '127.7866',
 '2019-10-31': '125.6243',
 '2019-09-30': '136.6057',
 '2019-08-30': '127.3152',
 '2019-07-31': '137.6629',
 '2019-06-28': '128.0607',
 '2019-05-31': '117.9291',
 '2019-04-30': '128.7208',
 '2019-03-29': '129.4825',
 '2019-02-28': '126.7570',
 '2019-01-31': '121.9154',
 '2018-12-31': '103.0957',
 '2018-11-30': '112.7096',
 '2018-10-31': '103.3765',
 '2018-09-28': '135.4203',
 '2018-08-31': '131.1842',
 '2018-07-31': '128.4035',
 '2018-06-29': '123.7699',
 '2018-05-31': '125.1963',
 '2018-04-30': '127.0316',
 '2018-03-29': '134.4541',
 '2018-02-28': '136.5572',
 